In [ ]:
from pyspark.sql import SparkSession

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("extract-cset-per-type") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("extract-cset-per-type").getOrCreate()

spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
from pyspark.sql import functions as f

from pyspark.sql.functions import split
from pyspark.sql.functions import col

readavant = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

csavant=readavant.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csavant.createOrReplaceTempView("CSET_avant")
csavant.show(truncate=150)

readapres = spark.read.option("header",True) \
  .csv("s3a://test-out/cset-wdc-2023-fix2/**")

# fix wrong formatting
csapres = csapres.withColumn("pset", f.regexp_replace(f.col("pset"), "isa:<([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?", "isa:<"))

csapres=readapres.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csapres.createOrReplaceTempView("CSET_apres")
csapres.show(truncate=150)

In [19]:
# calcule le count, l'average et le coverage d'un type de données
import json
import os
import re
def getCSETofType(data, pred, saveloc):
    
    
    # Danger injection SQL
    pred = pred.replace("'", "\\'")
    sets = spark.sql(f"SELECT pset, count FROM {data} WHERE pset LIKE '%{pred}%'")
    sets = sets.withColumn("pset", f.split(sets["pset"], " "))
    jsonsets = sets.toJSON().collect()
    
    pattern = r'isa:<schema\.org/(\w+)>'
    match = re.search(pattern, pred)
    typeN = match.group(1)
    
    if not os.path.exists(f"test-out/{saveloc}"):
        os.makedirs(f"test-out/{saveloc}")

    file_path = os.path.join(f"test-out/{saveloc}", f"{typeN}.json")
    
    with open(file_path, "w") as outfile:
        json.dump(jsonsets, outfile, indent=4)
    
    

In [20]:
from pyspark.sql import Row


type_name_list = 

print(type_name_list)

for pred in type_name_list:
    print(pred)

    getCSETofType("CSET_avant", pred, "cset-avant-per-type")
    getCSETofType("CSET_apres", pred, "cset-apres-per-type")
    

['isa:<schema.org/Recipe>']
isa:<schema.org/Recipe>
